In [1]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
from sklearn.model_selection import KFold
import os, json

import UPPtoolbox as upp


# Important variables
cwd = Path.cwd()
SNRs = np.array([-np.inf,-13,-11,-9,-7,-5,-3])
SubIDs = ['01','02','03','05','06','07','08','09','11','12','13','14','15','17','19','20','22','23','24','25']
colormap = {0: (0, 0, 0), 1: (0, 0.25, 1), 2: (0, 0.9375, 1), 3: (0, 0.91, 0.1), 4: (1, 0.6, 0), 5: (1, 0, 0), 6: (0.8, 0, 0)}

# Argument 1: The OU process fits perfectly

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from scipy import stats
from statsmodels.tsa.stattools import acf
from statsmodels.stats.stattools import durbin_watson
from tqdm.notebook import tqdm

global_results = []

for task in ['Active', 'Passive']:
    for period in ['late', 'early']:
        # Folder management
        save_path = Path("RestingStateResults") / task / period
        save_path.mkdir(parents=True, exist_ok=True)
        
        # Use Path.cwd() for cleaner path management
        base_dir = Path.cwd().parent 
        
        for part in tqdm(range(20), desc=f"Processing {task}-{period}"):
            # --- Data Loading ---
            data_ref = f'myEpochs_{task}/Epoch_{SubIDs[part]}-epo.fif'
            epochs_file = base_dir / data_ref 
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                tmin, tmax = (300, 500) if period == 'late' else (100, 200)
                # Ensure upp.STG is returning the expected tuple
                state_series_array = upp.STG(epochs_file, tmin=tmin, tmax=tmax)[0] 

            # Initialize model
            linear = upp.model.Linear(tau=10, process_noise=0.1, measure_noise=0.1, input_weight=0)
            
            # Fitting
            linear.fit(
                state_series={0: state_series_array},
                input_series={0: np.zeros_like(state_series_array)},
                init_params=[10, 0.1, 0.1, 0],
                bounds=[(1, 25), (0.01, 1), (0.01, 1), (0, 1)],
                fixed_params=['input_weight'],
                feedback=False)

            # Filtering
            f_series, m_noise, p_noise = linear.filter(
                state_series={0: state_series_array}, 
                input_series={0: np.zeros_like(state_series_array)}
            )
            
            # --- Analysis Logic ---
            m_noise_flat = m_noise.flatten()
            p_noise_flat = p_noise.flatten()
            
            # 1. Autocorrelation (10 Lags)
            n_lags = 10
            acf_m_matrix = np.array([acf(tr, nlags=n_lags, fft=True) for tr in m_noise])
            acf_p_matrix = np.array([acf(tr, nlags=n_lags, fft=True) for tr in p_noise])
            
            avg_acf_m = np.mean(acf_m_matrix, axis=0)
            avg_acf_p = np.mean(acf_p_matrix, axis=0)
            
            # 2. Durbin-Watson Test (Lag 1)
            dw_m = durbin_watson(m_noise_flat)
            dw_p = durbin_watson(p_noise_flat)

            # --- Visualization ---
            fig = plt.figure(figsize=(14, 10))
            plt.suptitle(f"Diagnostics: {task} {period} - Part {part}", fontsize=15, fontweight='bold')
            gs = fig.add_gridspec(3, 2, height_ratios=[1, 1, 1])

            # Row 1: Tracking Illustration
            ax1 = fig.add_subplot(gs[0, :])
            ax1.plot(state_series_array[0], label='Observed (Trial 1)', color='black', alpha=0.3)
            ax1.plot(f_series[0], label='OU Filtered', color='tab:blue', lw=1.5)
            ax1.set_title("Filter Tracking (Single Trial Example)")
            ax1.legend(loc='upper right')

            # Row 2: Gaussianity Plots
            for i, (data, title, col) in enumerate([
                (m_noise_flat, "Measurement Noise", "tab:green"),
                (p_noise_flat, "Process Noise", "tab:orange")
            ]):
                ax = fig.add_subplot(gs[1, i])
                # Plot Histogram
                sns.histplot(data, ax=ax, color=col, stat="density", alpha=0.4)
                
                # Fit Gaussian Curve
                mu, std = stats.norm.fit(data)
                x = np.linspace(data.min(), data.max(), 100)
                p = stats.norm.pdf(x, mu, std)
                ax.plot(x, p, 'k', lw=2, label=f'Fit: μ={mu:.2f}, σ={std:.2f}')
                ax.set_title(f"{title} Distribution")
                ax.legend(fontsize='small')

            # Row 3: Whiteness (ACF 10 Lags)
            ci = 1.96 / np.sqrt(state_series_array.shape[1])
            
            # Measurement ACF Plot
            ax4 = fig.add_subplot(gs[2, 0])
            ax4.stem(range(n_lags + 1), avg_acf_m)
            ax4.axhline(ci, color='tab:red', ls='--', alpha=0.5, label='95% CI')
            ax4.axhline(-ci, color='tab:red', ls='--', alpha=0.5)
            ax4.set_title(f"Measure Noise ACF (Durbin-Watson={dw_m:.2f})")
            ax4.set_ylim([-0.2, 1.1])

            # Process ACF Plot
            ax5 = fig.add_subplot(gs[2, 1])
            ax5.stem(range(n_lags + 1), avg_acf_p)
            ax5.axhline(ci, color='tab:red', ls='--', alpha=0.5)
            ax5.axhline(-ci, color='tab:red', ls='--', alpha=0.5)
            ax5.set_title(f"Process Noise ACF (Durbin-Watson={dw_p:.2f})")
            ax5.set_ylim([-0.2, 1.1])

            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.savefig(save_path / f"part{part}_figure.png", dpi=150)
            plt.close(fig)

Processing Active-late:   0%|          | 0/20 [00:00<?, ?it/s]

Processing Active-early:   0%|          | 0/20 [00:00<?, ?it/s]

Processing Passive-late:   0%|          | 0/20 [00:00<?, ?it/s]

Processing Passive-early:   0%|          | 0/20 [00:00<?, ?it/s]

# Argument 2: Model fitting on resting-state only trials